# Scrap My Prop

### Laboratórios de Engenharia Informática

#### "Development of an IT solution for the extraction and automatic analysis of data and relevant information for the calculation of land and properties."

## Importar Bibliotecas Python

In [ ]:
import pandas as pd
import numpy as np
import re

# 'ERA'

## Ler Dados do CSV

In [ ]:
data_era = pd.read_csv('dados/dados_era_01_04.csv')
pd.set_option('display.max_columns', None)
data_era

## Pré-processamento comum a todo o Dataset

** Funções auxiliares **

In [ ]:
def ajustPreco(string):
    return re.sub(r"(.*?)€",r"\1",string)

** Colunas 'Distrito' e 'Concelho': Retirar colunas de forma total porque já era uma pré-condição do projeto, mas primeiro verificar se, de facto, todos os campos estão preenchidos com 'Braga'; **

In [ ]:
print("Distritos existentes:", data_era['Distrito'].unique())
print("Concelhos existentes:", data_era['Concelho'].unique())

In [ ]:
if 'Distrito' in data_era.columns:
    data_era = data_era.drop(['Distrito'], axis = 1) 
if 'Concelho' in data_era.columns:
    data_era = data_era.drop(['Concelho'], axis = 1) 

** Colunas 'Nome' e 'Referência': Não são características dos imóveis mas sim identificadores, portanto não devem entrar nos cálculos (Drop); **

In [ ]:
if 'Nome' in data_era.columns:
    data_era = data_era.drop(['Nome'], axis = 1) 
if 'Referência' in data_era.columns:
    data_era = data_era.drop(['Referência'], axis = 1) 
data_era.info()

** Coluna 'Preço de Venda': Drop dos imóveis que estão 'Sob Consulta'; Retirar símbolo '€'; **

In [ ]:
data_era = data_era.drop(data_era[(data_era['Preço de Venda'] == 'Sob Consulta')].index)
data_era.index = np.arange(1, len(data_era) + 1)
data_era['Preço de Venda'] = data_era['Preço de Venda'].apply(ajustPreco)
data_era

** Coluna 'Freguesia': Não podem existir valores nulos nesta coluna, senão estamos a perder o principal fator de diferenciação dos preços; **

In [ ]:
data_era['Freguesia'].isnull().values.any()

** Coluna 'Estado': 'N/D' deve passar a notação que seja interpretada pelas bibliotecas do Python ('NaN') **

In [ ]:
data_era = data_era.replace('N/D',np.nan)
data_era

In [ ]:
data_era.info()

!!! FALTA VER COMO TRATAR DAS FEATURES ZONA E ETC !!!

## Pré-processamento diferenciado por tipologia

** Listar todas as tipologias existentes **

In [ ]:
data_era['Tipo de Imóvel'].unique()

** Dividir os dados por tipologia **

In [ ]:
data_era_moradias = data_era.loc[(data_era['Tipo de Imóvel'] == 'Moradia em Banda') | (data_era['Tipo de Imóvel'] == 'Moradia Geminada') | (data_era['Tipo de Imóvel'] == 'Moradia Isolada') | (data_era['Tipo de Imóvel'] == 'Moradia') | (data_era['Tipo de Imóvel'] == 'Andar Moradia') | (data_era['Tipo de Imóvel'] == 'Moradia Rústica')]
data_era_moradias.index = np.arange(1, len(data_era_moradias) + 1)

data_era_apartamentos = data_era.loc[(data_era['Tipo de Imóvel'] == 'Apartamento') | (data_era['Tipo de Imóvel'] == 'Duplex')]
data_era_apartamentos.index = np.arange(1, len(data_era_apartamentos) + 1)

data_era_terrenos = data_era.loc[(data_era['Tipo de Imóvel'] == 'Lote') | (data_era['Tipo de Imóvel'] == 'Terreno Para Construção') | (data_era['Tipo de Imóvel'] == 'Terreno com ruina') | (data_era['Tipo de Imóvel'] == 'Terreno Rústico') | (data_era['Tipo de Imóvel'] == 'Lote Industrial') | (data_era['Tipo de Imóvel'] == 'Terreno') | (data_era['Tipo de Imóvel'] == 'Loteamento') | (data_era['Tipo de Imóvel'] == 'Terreno Misto')]
data_era_terrenos.index = np.arange(1, len(data_era_terrenos) + 1)

data_era_lojas = data_era.loc[(data_era['Tipo de Imóvel'] == 'Loja')]
data_era_lojas.index = np.arange(1, len(data_era_lojas) + 1)

# faltam tipologias

## Dados - Moradias

In [ ]:
data_era_moradias.head()

In [ ]:
data_era_moradias.info()

** Dados em falta por linha **

In [ ]:
percent_missing = (1 - data_era_moradias.apply(lambda x: x.count(), axis=1) / len(data_era_moradias.columns)) * 100
missing_value_data_era_moradias_rows = pd.DataFrame({'percent_missing (%)': percent_missing})
missing_value_data_era_moradias_rows.sort_values('percent_missing (%)', inplace=True, ascending=False)
missing_value_data_era_moradias_rows

** Dados em falta por coluna **

In [ ]:
percent_missing = data_era_moradias.isnull().sum() * 100 / len(data_era_moradias)
missing_value_data_era_moradias_columns = pd.DataFrame({'percent_missing (%)': percent_missing})
missing_value_data_era_moradias_columns.sort_values('percent_missing (%)', inplace=True, ascending=False)
missing_value_data_era_moradias_columns
